In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [2]:
# Scale the pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
# Convert the labels to one-hot encoded vectors
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
# Reshape
x_train = x_train.reshape(x_train.shape[0], 784)
x_test = x_test.reshape(x_test.shape[0], 784)
# Split data into 90% training and 10% validation sets
split = int(0.9 * len(x_train))
x_train, x_val = x_train[:split], x_train[split:]
y_train, y_val = y_train[:split], y_train[split:]

In [4]:
# Define MLP model architecture
model = keras.Sequential([
    keras.layers.Dense(units=256, activation='relu', input_shape=(784,)),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Define tensorboard logs directory
log_dir="C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP"
tensorboard = TensorBoard(log_dir='C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP', histogram_freq=1, write_graph=True, write_images=False)

In [6]:
# Train the model
model.fit(x_train, y_train, epochs=30, batch_size=128, validation_data=(x_val, y_val), callbacks=[tensorboard])

Epoch 1/30
422/422 [==============================] - 8s 13ms/step - loss: 0.5287 - accuracy: 0.8129 - val_loss: 0.4005 - val_accuracy: 0.8545
Epoch 2/30
422/422 [==============================] - 4s 10ms/step - loss: 0.3778 - accuracy: 0.8647 - val_loss: 0.3589 - val_accuracy: 0.8673
Epoch 3/30
422/422 [==============================] - 4s 10ms/step - loss: 0.3298 - accuracy: 0.8790 - val_loss: 0.3504 - val_accuracy: 0.8742
Epoch 4/30
422/422 [==============================] - 5s 11ms/step - loss: 0.3051 - accuracy: 0.8880 - val_loss: 0.3341 - val_accuracy: 0.8713
Epoch 5/30
422/422 [==============================] - 8s 19ms/step - loss: 0.2849 - accuracy: 0.8954 - val_loss: 0.3148 - val_accuracy: 0.8827
Epoch 6/30
422/422 [==============================] - 6s 14ms/step - loss: 0.2743 - accuracy: 0.8988 - val_loss: 0.3167 - val_accuracy: 0.8837
Epoch 7/30
422/422 [==============================] - 5s 12ms/step - loss: 0.2578 - accuracy: 0.9034 - val_loss: 0.3271 - val_accuracy: 0.8812

In [11]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Generate predictions on the test set
y_pred = model.predict(x_test)

# Convert predicted probabilities to predicted class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert true labels to one-hot encoding
y_true = np.argmax(y_test, axis=1)

313/313 [==============================] - 1s 3ms/step


In [8]:
# Compute evaluation metrics
conf_matrix = confusion_matrix(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='macro')
recall = recall_score(y_true, y_pred_classes, average='macro')
f1score = f1_score(y_true, y_pred_classes, average='macro')
auc_roc = roc_auc_score(y_test, y_pred, multi_class='ovr')

# Print the evaluation metrics
print("Test Loss:", test_loss)
print("Accuracy:", test_acc)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)
print("AU-ROC:", auc_roc)

Test Loss: 0.4242440462112427
Accuracy: 0.8909000158309937
Confusion Matrix:
 [[868   3  17  20   4   2  81   0   5   0]
 [  3 982   0   9   2   0   3   0   1   0]
 [ 19   1 852   9  70   0  48   0   1   0]
 [ 24  13  11 890  41   0  17   0   4   0]
 [  0   0 108  22 792   0  77   0   1   0]
 [  0   0   0   1   0 963   0  20   1  15]
 [121   0  99  29  47   1 696   0   7   0]
 [  0   0   0   0   0  13   0 979   1   7]
 [  5   0   6   5   5   4   7   3 965   0]
 [  0   0   0   1   0   7   1  69   0 922]]
Precision: 0.8914492833855905
Recall: 0.8909
F1-score: 0.8906521077304161
AU-ROC: 0.9905555611111112


In [9]:
# Log evaluation metrics to TensorBoard
test_writer = tf.summary.create_file_writer(log_dir)
with test_writer.as_default():
    tf.summary.scalar('Accuracy', test_acc, step=0)
    tf.summary.scalar('Precision', precision, step=0)
    tf.summary.scalar('Recall', recall, step=0)
    tf.summary.scalar('F1-score', f1score, step=0)
    tf.summary.scalar('AU-ROC', auc_roc, step=0)

In [10]:
# tensorboard --logdir_spec=FashionMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP",FashionLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionLeNet",MnistLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistLeNet",MnistMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistMLP"